In [ ]:
MySQL Config Editor 
=============================

-- helps in storing encrypted database credentials.
-- with filname .mylogin.cnf in users home directory.
-- default group is [client] and for any other group --login-path has to be specicified.


Set with default group 
===============================

[mysqluser@mysqlserver ~]$ mysql_config_editor set -u root -p
Enter password: 

[mysqluser@mysqlserver ~]$ mysql_config_editor print
[client]
user = "root"
password = *****

[mysqluser@mysqlserver ~]$ pwd
/home/mysqluser

[mysqluser@mysqlserver ~]$ ls -artl .mylogin.cnf
-rw-------. 1 mysqluser mysqluser 100 May 16 00:21 .mylogin.cnf
[mysqluser@mysqlserver ~]$ 


[mysqluser@mysqlserver ~]$ mysql

Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 8

mysql> select user();
+----------------+
| user()         |
+----------------+
| root@localhost |
+----------------+
1 row in set (0.00 sec)



Set with dbadmin_client group 
===========================================

[mysqluser@mysqlserver ~]$ mysql_config_editor set --login-path=dbadmin_client -u db_admin -p
Enter password: 
[mysqluser@mysqlserver ~]$ 


[mysqluser@mysqlserver ~]$ mysql_config_editor print
[client]
user = "root"
password = *****


[mysqluser@mysqlserver ~]$ mysql_config_editor print --all
[client]
user = "root"
password = *****
[dbadmin_client]
user = "db_admin"
password = *****


[mysqluser@mysqlserver ~]$ mysql --login-path=dbadmin_client

Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 9

mysql> select user();
+--------------------+
| user()             |
+--------------------+
| db_admin@localhost |
+--------------------+
1 row in set (0.00 sec)



Remove/Reset all the stored credentials
==================================================

[mysqluser@mysqlserver ~]$ mysql_config_editor reset
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysql_config_editor print --all
[mysqluser@mysqlserver ~]$ 

In [ ]:
Mysqladmin Program
===================================

-- is a program to carry out admin tasks like shutdown , create/drop database , check status , ping , 
   start/stop replica.


[mysqluser@mysqlserver ~]$ mysqladmin ping
mysqld is alive

[mysqluser@mysqlserver ~]$ mysqladmin status
Uptime: 1621  Threads: 2  Questions: 10  Slow queries: 0  Opens: 119  Flush tables: 3  Open tables: 38 
 Queries per second avg: 0.006

[mysqluser@mysqlserver ~]$ mysqladmin flush-tables 
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysqladmin status
Uptime: 1668  Threads: 2  Questions: 14  Slow queries: 0  Opens: 119  Flush tables: 4  Open tables: 0  
Queries per second avg: 0.008

[mysqluser@mysqlserver ~]$ mysqladmin shutdown 
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysqladmin ping
mysqladmin: connect to server at 'localhost' failed
Check that mysqld is running and that the socket: '/var/lib/mysql/mysql.sock' exists!

[mysqluser@mysqlserver ~]$ systemctl start mysqld
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysqladmin ping
mysqld is alive

[mysqluser@mysqlserver ~]$ mysqladmin create employeesdb
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysql
Welcome to the MySQL monitor.  Commands end with ; or \g.

mysql> show databases like '%emp%';
+------------------+
| Database (%emp%) |
+------------------+
| employeesdb      |
+------------------+

[mysqluser@mysqlserver ~]$ mysqladmin drop employeesdb

Do you really want to drop the 'employeesdb' database [y/N] y

Database "employeesdb" dropped

[mysqluser@mysqlserver ~]$ 


[mysqluser@mysqlserver ~]$ mysql
Welcome to the MySQL monitor.  Commands end with ; or \g.

mysql> show databases like '%emp%';
Empty set (0.00 sec)

In [ ]:
MySQL Executing SQL Files 
==========================================


using source or \. 
=============================

[mysqluser@mysqlserver ~]$ cat create_table_staff.sql 
use employeesdb;
create table staff (
	staff_id int primary key,
	staff_name varchar(50)
	staff_email varchar(100)
);
[mysqluser@mysqlserver ~]$ 


mysql> source create_table_staff.sql
Database changed
Query OK, 0 rows affected (0.06 sec)

mysql> show tables;
+-----------------------+
| Tables_in_employeesdb |
+-----------------------+
| staff                 |
+-----------------------+

mysql> drop table staff;
Query OK, 0 rows affected (0.01 sec)

mysql> \. create_table_staff.sql
Database changed
Query OK, 0 rows affected (0.03 sec)

mysql> show tables;
+-----------------------+
| Tables_in_employeesdb |
+-----------------------+
| staff                 |
+-----------------------+



using < operator
==================================

[mysqluser@mysqlserver ~]$ mysql < create_table_staff.sql 
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysql

Welcome to the MySQL monitor.  Commands end with ; or \g.

mysql> use employeesdb;

Database changed

mysql> show tables;
+-----------------------+
| Tables_in_employeesdb |
+-----------------------+
| staff                 |
+-----------------------+
1 row in set (0.00 sec)


using bash script 
======================================

[mysqluser@mysqlserver ~]$ vi create_table_staff.sh
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ cat create_table_staff.sh
mysql < $1
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ chmod 755 create_table_staff.sh
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ ./create_table_staff.sh create_table_staff.sql &
[1] 4661
[mysqluser@mysqlserver ~]$ 
[1]+  Done                    ./create_table_staff.sh create_table_staff.sql
[mysqluser@mysqlserver ~]$ 


pipe method 
=========================

[mysqluser@mysqlserver ~]$ cat create_table_staff.sql | mysql
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysql
Welcome to the MySQL monitor.  Commands end with ; or \g.

mysql> use employeesdb;

Database changed

mysql> show tables;
+-----------------------+
| Tables_in_employeesdb |
+-----------------------+
| staff                 |
+-----------------------+
1 row in set (0.00 sec)

In [ ]:
mysqlimport program
===================================

-- is a data import program , takes .txt file as input with tab delimited and table name as filename.
-- can be loaded only with location of secure_file_priv variable.

[mysqluser@mysqlserver ~]$ vi staff_data.txt
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ cat staff_data.txt 
1001 "Arvind" "gsddd@dfff.com"
1002 "Aarna" "hbchbcc@vfff.com"
1003 "Meghana" "xcxgsgfd@gdhad.com"
[mysqluser@mysqlserver ~]$ 


[mysqluser@mysqlserver ~]$ mysqlimport employeesdb staff_data.txt 

mysqlimport: Error: 1146, Table 'employeesdb.staff_data' doesn't exist, when using table: staff_data

[mysqluser@mysqlserver ~]$ 


[mysqluser@mysqlserver ~]$ mv staff_data.txt staff.txt
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysqlimport employeesdb staff.txt

mysqlimport: Error: 1290, The MySQL server is running with the --secure-file-priv option so it cannot 
execute this statement, when using table: staff


mysql> show variables like '%secure%';
+--------------------------+-----------------------+
| Variable_name            | Value                 |
+--------------------------+-----------------------+
| require_secure_transport | OFF                   |
| secure_file_priv         | /var/lib/mysql-files/ |
+--------------------------+-----------------------+
2 rows in set (0.00 sec)

[root@mysqlserver mysqluser]# mv staff.txt /var/lib/mysql-files/

[root@mysqlserver mysql-files]# chown mysql:mysql staff.txt 
[root@mysqlserver mysql-files]# 

[mysqluser@mysqlserver ~]$ mysqlimport employeesdb /var/lib/mysql-files/staff.txt
mysqlimport: Error: 1265, Data truncated for column 'staff_id' at row 1, when using table: staff
[mysqluser@mysqlserver ~]$ 

proper tab limited text file 
================================

[root@mysqlserver mysql-files]# vi staff.txt

[root@mysqlserver mysql-files]# cat staff.txt 
1001	Arvind Kamath	gdgfgff@fdff.com
1002	Aarna Kamath	fsdfsgdsdd@vafff.com
1003	Meghan Kamath	vxcxcxcv@hgfdad.com
[root@mysqlserver mysql-files]# 


[root@mysqlserver mysql-files]# mysqlimport -u root -p employeesdb /var/lib/mysql-files/staff.txt
Enter password: 
employeesdb.staff: Records: 3  Deleted: 0  Skipped: 0  Warnings: 0
[root@mysqlserver mysql-files]# 

mysql> select * from staff;
+----------+---------------+----------------------+
| staff_id | staff_name    | staff_email          |
+----------+---------------+----------------------+
|     1001 | Arvind Kamath | gdgfgff@fdff.com     |
|     1002 | Aarna Kamath  | fsdfsgdsdd@vafff.com |
|     1003 | Meghan Kamath | vxcxcxcv@hgfdad.com  |
+----------+---------------+----------------------+
3 rows in set (0.00 sec)



mysql> create table employees (
    -> emp_id int auto_increment primary key,
    -> emp_name varchar(50),
    -> emp_phone varchar(50),
    -> emp_email varchar(50)
    -> );
Query OK, 0 rows affected (0.04 sec)

[root@mysqlserver mysql-files]# mysqlimport -u root -p employeesdb /var/lib/mysql-files/employees.txt 
Enter password: 
mysqlimport: Error: 1366, Incorrect integer value: 'Ranganna' for column 'emp_id' at row 1, when using table: employees
[root@mysqlserver mysql-files]# 

[root@mysqlserver mysql-files]# mysqlimport -u root -p -c emp_name,emp_phone,emp_email employeesdb /var/lib/mysql-files/employees.txt
Enter password: 
employeesdb.employees: Records: 3  Deleted: 0  Skipped: 0  Warnings: 0
[root@mysqlserver mysql-files]# 

In [ ]:
mysqlcheck program
=======================================

-- is a table maintenence program , it checks , repairs , oprtimize or analyse the table.
-- table will be locked during the opertion.
-- -c check , -a analyze , -o optimize , -r repair.
-- -c is default.

[mysqluser@mysqlserver ~]$ mysqlcheck employeesdb employees
employeesdb.employees                              OK
[mysqluser@mysqlserver ~]$ 
                          
[mysqluser@mysqlserver ~]$ mysqlcheck -o -a employeesdb employees
Error:  mysqlcheck doesn't support multiple contradicting commands.
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysqlcheck -o employeesdb employees
employeesdb.employees
note     : Table does not support optimize, doing recreate + analyze instead
status   : OK

[mysqluser@mysqlserver ~]$ mysqlcheck -r employeesdb employees
employeesdb.employees
note     : The storage engine for the table doesn't support repair

[mysqluser@mysqlserver ~]$ mysqlcheck -a employeesdb employees
employeesdb.employees                              OK


simulate data corruption by editing the os table file
==========================================================

[root@mysqlserver employeesdb]# vi employees.ibd 
[root@mysqlserver employeesdb]# 
[root@mysqlserver employeesdb]# ls -rtl
total 0
-rw-r-----. 1 mysql mysql 0 May 16 19:11 employees.ibd
[root@mysqlserver employeesdb]# 

[mysqluser@mysqlserver ~]$ mysqlcheck -v -c employeesdb employees
# Connecting to localhost...
employeesdb.employees                              OK
# Disconnecting from localhost...

[root@mysqlserver employeesdb]# ls -rtl
total 0
-rw-r-----. 1 mysql mysql 0 May 16 19:11 employees.ibd
[root@mysqlserver employeesdb]# 

[root@mysqlserver employeesdb]# mysqlcheck -u root -p -v -r employeesdb employees
# Connecting to localhost...
Enter password: 
employeesdb.employees
note     : The storage engine for the table doesn't support repair
# Disconnecting from localhost...
[root@mysqlserver employeesdb]# 

[root@mysqlserver employeesdb]# mysqlcheck -u root -p -o employeesdb employees
Enter password: 
employeesdb.employees
note     : Table does not support optimize, doing recreate + analyze instead
status   : OK
[root@mysqlserver employeesdb]# 
[root@mysqlserver employeesdb]# ls -rtl
total 112
-rw-r-----. 1 mysql mysql 114688 May 16 19:17 employees.ibd
[root@mysqlserver employeesdb]# 

mysql> select * from employeesdb.employees;
+--------+----------+-----------+----------------+
| emp_id | emp_name | emp_phone | emp_email      |
+--------+----------+-----------+----------------+
|      3 | rama     | 67545     | ttrr@vcc.com   |
|      4 | bhima    | 65457     | gdfff@jhhf.com |
|      5 | shama    | 54464     | dffdff@gdd.com |
|      6 | rama     | 67545     | ttrr@vcc.com   |
|      7 | bhima    | 65457     | gdfff@jhhf.com |
|      8 | shama    | 54464     | dffdff@gdd.com |
+--------+----------+-----------+----------------+
6 rows in set (0.00 sec)


simulate data corruption by editing the os table file , if server restarted the table cannot be recreated
============================================================================================================

[root@mysqlserver employeesdb]# vi staff.ibd 
[root@mysqlserver employeesdb]# 
[root@mysqlserver employeesdb]# ls -rtl
total 112
-rw-r-----. 1 mysql mysql 114688 May 16 02:33 employees.ibd
-rw-r-----. 1 mysql mysql      0 May 16 02:36 staff.ibd
[root@mysqlserver employeesdb]# 

[root@mysqlserver employeesdb]# systemctl restart mysqld
[root@mysqlserver employeesdb]# 

[root@mysqlserver employeesdb]# mysqlcheck -u root -p -c employeesdb staff
Enter password: 

mysqlcheck: Got error: 2013: Lost connection to MySQL server during query when executing 'CHECK TABLE ... '

[root@mysqlserver employeesdb]# ls -rtl
total 112
-rw-r-----. 1 mysql mysql 114688 May 16 02:33 employees.ibd
-rw-r-----. 1 mysql mysql      0 May 16 02:36 staff.ibd
[root@mysqlserver employeesdb]# 

mysql> select * from staff;
ERROR 1812 (HY000): Tablespace is missing for table `employeesdb`.`staff`.
mysql> 

In [ ]:
mysqlshow program
================================

-- displays database , table and column information.

[mysqluser@mysqlserver ~]$ mysqlshow employeesdb

Database: employeesdb
+-----------+
|  Tables   |
+-----------+
| employees |
| staff     |
+-----------+


[mysqluser@mysqlserver ~]$ mysqlshow employeesdb employees

Database: employeesdb  Table: employees
+-----------+-------------+--------------------+------+-----+---------+----------------+---------------------------------+---------+
| Field     | Type        | Collation          | Null | Key | Default | Extra          | Privileges                      | Comment |
+-----------+-------------+--------------------+------+-----+---------+----------------+---------------------------------+---------+
| emp_id    | int         |                    | NO   | PRI |         | auto_increment | select,insert,update,references |         |
| emp_name  | varchar(50) | utf8mb4_0900_ai_ci | YES  |     |         |                | select,insert,update,references |         |
| emp_phone | varchar(50) | utf8mb4_0900_ai_ci | YES  |     |         |                | select,insert,update,references |         |
| emp_email | varchar(50) | utf8mb4_0900_ai_ci | YES  |     |         |                | select,insert,update,references |         |
+-----------+-------------+--------------------+------+-----+---------+----------------+---------------------------------+---------+


[mysqluser@mysqlserver ~]$ mysqlshow employeesdb employees emp_id

Database: employeesdb  Table: employees  Wildcard: emp_id
+--------+------+-----------+------+-----+---------+----------------+---------------------------------+---------+
| Field  | Type | Collation | Null | Key | Default | Extra          | Privileges                      | Comment |
+--------+------+-----------+------+-----+---------+----------------+---------------------------------+---------+
| emp_id | int  |           | NO   | PRI |         | auto_increment | select,insert,update,references |         |
+--------+------+-----------+------+-----+---------+----------------+---------------------------------+---------+
[mysqluser@mysqlserver ~]$ 


In [ ]:
mysql_tzinfo_to_sql program
=======================================

-- loads time zone data from zoneinfo database into mysql database.
-- location in linux is /usr/share/zoneinfo 


[mysqluser@mysqlserver ~]$ ls -rtl /usr/share/zoneinfo/
total 392
-rw-r--r--.  1 root root   4463 May 10  2021 iso3166.tab
-rw-r--r--.  1 root root   3388 Jul 22  2021 leapseconds
-rw-r--r--.  1 root root  19419 Sep 20  2021 zone.tab
-rw-r--r--.  1 root root  17593 Sep 25  2021 zone1970.tab
-rw-r--r--.  8 root root    127 Oct 26  2021 Zulu
-rw-r--r--.  2 root root   1544 Oct 26  2021 W-SU
-rw-r--r--.  1 root root   1873 Oct 26  2021 WET
-rw-r--r--.  8 root root    127 Oct 26  2021 UTC
-rw-r--r--.  8 root root    127 Oct 26  2021 Universal
-rw-r--r--.  8 root root    127 Oct 26  2021 UCT


mysql> use mysql

Database changed

mysql> show tables like 'time_zone%';

+------------------------------+
| Tables_in_mysql (time_zone%) |
+------------------------------+
| time_zone                    |
| time_zone_leap_second        |
| time_zone_name               |
| time_zone_transition         |
| time_zone_transition_type    |
+------------------------------+


mysql> select count(*) from time_zone;

+----------+
| count(*) |
+----------+
|        0 |
+----------+

mysql> select count(*) from time_zone_name;

+----------+
| count(*) |
+----------+
|        0 |
+----------+

[mysqluser@mysqlserver ~]$ mysql_tzinfo_to_sql /usr/share/zoneinfo/ | mysql mysql
Warning: Unable to load '/usr/share/zoneinfo//iso3166.tab' as time zone. Skipping it.
Warning: Unable to load '/usr/share/zoneinfo//leapseconds' as time zone. Skipping it.
Warning: Unable to load '/usr/share/zoneinfo//tzdata.zi' as time zone. Skipping it.
Warning: Unable to load '/usr/share/zoneinfo//zone.tab' as time zone. Skipping it.
Warning: Unable to load '/usr/share/zoneinfo//zone1970.tab' as time zone. Skipping it.
[mysqluser@mysqlserver ~]$ 


mysql> use mysql;

Database changed

mysql> select count(*) from time_zone;

+----------+
| count(*) |
+----------+
|     1783 |
+----------+

mysql> select count(*) from time_zone_name;

+----------+
| count(*) |
+----------+
|     1783 |
+----------+